In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train_V2.csv', 'test_V2.csv', 'sample_submission_V2.csv']


In [16]:
data=pd.read_csv('../input/train_V2.csv')
data = data.drop(data[((data.rankPoints != -1) & (data.winPoints==0)) | (data.rankPoints == -1) ].index)

data=data.drop(['rankPoints'],axis=1) # this feature is inconsistent and thus removed for analysis
#print (len(data))
data=data.dropna()
#print (len(data))



solo_data=data.loc[data['matchType'] == 'solo']
solo_data=solo_data.drop(['matchType'],axis=1)

from sklearn.preprocessing import StandardScaler
# all_colns=list(data.columns.values)
# all_colns.remove('Id')
# all_colns.remove('groupId')
# all_colns.remove('matchId')
# all_colns.remove('matchType')
# all_colns.remove('winPoints')
# all_colns.remove('winPlacePerc')
# #print(all_colns)
# cols_to_norm=all_colns
# solo_data[cols_to_norm] = StandardScaler().fit_transform(solo_data[cols_to_norm])






solo_train=solo_data.sample(frac=0.8)
solo_test=solo_data.drop(solo_train.index)
x_train=solo_train.iloc[:,3:-1]
y_train=solo_train.iloc[:,-1]
x_test=solo_test.iloc[:,3:-1]
y_test=solo_test.iloc[:,-1]


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RFR
# 'max_depth':np.arange(1,10,1.5)}
original_param_grid = [
  {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
original_RF=RFR()
original_RF_CV=GridSearchCV(original_RF,original_param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
original_RF_CV.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]), 'max_features': ['auto', 'sqrt', 'log2']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
from sklearn.metrics import mean_absolute_error as mae
original_predict=original_RF_CV.predict(x_test)
mae(y_test,original_predict)

0.04695315466462144

In [19]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error as mae
elo_x_train=x_train.iloc[:,:-1]
elo_y_train=x_train.iloc[:,-1]
elo_x_test=x_test.iloc[:,:-1]
elo_y_test=x_test.iloc[:,-1]


elo_param_grid = [
  {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
elo_RF=RFR()
elo_RF_CV=GridSearchCV(elo_RF,elo_param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
elo_RF_CV.fit(elo_x_train,elo_y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]), 'max_features': ['auto', 'sqrt', 'log2']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [20]:
from sklearn.metrics import mean_absolute_error as mae
print ('elo prediction score',elo_RF_CV.score(elo_x_test,elo_y_test))

elo prediction score 0.5559786698214774


In [25]:
elo_diff=np.sign(elo_RF_CV.predict(elo_x_train)-elo_y_train)
new_x_train=pd.concat([x_train,elo_diff],axis=1)
param_grid = [
  {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
RF=RFR()
RF_CV=GridSearchCV(RF,param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
RF_CV.fit(new_x_train,y_train)

test_elo_diff=np.sign(elo_RF_CV.predict(elo_x_test)-elo_y_test)
new_x_test=pd.concat([x_test,test_elo_diff],axis=1)
mae(y_test,RF_CV.predict(new_x_test))

0.046879229399576276

In [21]:
# elo*final_placement  , this is a trial feature#   bad performance ----> abandoned

# train the model to predict the in-game performance
in_game_score=elo_y_train*y_train

performance_param_grid = [
  {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
performance_RF=RFR()
performance_RF_CV=GridSearchCV(performance_RF,performance_param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
performance_RF_CV.fit(elo_x_train,in_game_score)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]), 'max_features': ['auto', 'sqrt', 'log2']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [23]:
# using in-game performance as features
performance_x_train=pd.concat([x_train,in_game_score],axis=1)
performance_param_grid = [
  {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
RF=RFR()
Test_clf=GridSearchCV(RF,performance_param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
Test_clf.fit(performance_x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100]), 'max_features': ['auto', 'sqrt', 'log2']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [24]:
added=performance_RF_CV.predict(elo_x_test)
added=pd.DataFrame(added)

in_game_score_x_test=pd.concat([x_test.reset_index(),added],axis=1)
in_game_score_x_test=in_game_score_x_test.iloc[:,1:]
mae(y_test,Test_clf.predict(in_game_score_x_test))

0.04920694835401528

In [ ]:
#### above result is worse than original random forest

In [ ]:
# multivariate polynomial regressor to predict ELO   ##### bad performance on prediction on ELO
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(elo_x_train)
#elo_y_train=np.array(elo_y_train)
predict_ = poly.fit_transform(elo_x_test)
clf = LinearRegression()
clf.fit(X_, elo_y_train)
clf.score(predict_,elo_y_test)

In [ ]:
##################################################################################
##################################################################################
##################################################################################

#       This is ELO difference predictor  
#       This is ELO difference predictor
#       This is ELO difference predictor  

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import GridSearchCV
def elo_predictor(train,test): # last column of df should be winPoints, train is used to train elo_predictor
    
    train = train.drop(train[((train.rankPoints != -1) & (train.winPoints==0)) | (train.rankPoints == -1) ].index)
    train= train.drop(['rankPoints'],axis=1) # this feature is inconsistent and thus removed for analysis
    
    test = test.drop(test[((test.rankPoints != -1) & (test.winPoints==0)) | (test.rankPoints == -1) ].index)
    test = test.drop(['rankPoints'],axis=1) # this feature is inconsistent and thus removed for analysis
    
    x_train=train.iloc[:,3:-1]   # First three columns are ID information, not useful for prediction and thus removed
    y_train=train.iloc[:,-1]     # Matchtype should be preprocessed by one-hot-encoding before passing into function
    x_test=test.iloc[:,3:-1]     # Both input should include rankPoints. They will be dropped later in this function
    y_test=test.iloc[:,-1]
    
    
#     x_train = x_train.drop(x_train[((x_train.rankPoints != -1) & (x_train.winPoints==0)) | (x_train.rankPoints == -1) ].index)
#     x_train= x_train.drop(['rankPoints'],axis=1) # this feature is inconsistent and thus removed for analysis
    
#     x_test = x_test.drop(x_test[((x_test.rankPoints != -1) & (x_test.winPoints==0)) | (x_test.rankPoints == -1) ].index)
#     x_test = x_test.drop(['rankPoints'],axis=1) # this feature is inconsistent and thus removed for analysis
    
#     elo_x_train=x_train.iloc[:,:-1]
#     elo_y_train=train.iloc[:,-1]
#     elo_x_test=test.iloc[:,:-1]
#     elo_y_test=test.iloc[:,-1]
    elo_param_grid = [
      {'n_estimators': np.arange(10,110,10), 'max_features': ['auto','sqrt','log2']}]
    elo_RF=RFR()
    elo_RF_CV=GridSearchCV(elo_RF,elo_param_grid,cv=5,refit=True,return_train_score=True,n_jobs=-1)
    elo_RF_CV.fit(x_train,y_train)
    elo_predict=elo_RF_CV.predict(x_test)
    elo_diff=elo_predict-y_test   # postiive = player is doing better than before, negative = playis is doing worse than before
    return (elo_diff)
    
    

In [ ]:
#############  test function
data=pd.read_csv('../input/train_V2.csv')
a=data.sample(frac=0.1)

b=data.sample(frac=0.1)

# a=pd.Series.to_frame(a)
# b=pd.Series.to_frame(b)
a['matchType']=a['matchType'].astype('category').cat.codes
b['matchType']=b['matchType'].astype('category').cat.codes
elo_predictor(a,b)